In [1]:
import cv2 as cv
import numpy as np
import os
from operator import itemgetter

In [2]:
def show_image(title, image):
    image = cv.resize(image, (0,0), fx=0.25, fy=0.25)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def extrage_careu(image):
    original = image.copy()
    
    low_yellow = (20, 0, 0)
    high_yellow = (130, 255, 255)
    img_hsv = cv.cvtColor(image , cv.COLOR_BGR2HSV)
    image = cv.inRange(img_hsv, low_yellow, high_yellow)
    
    image_median_blur = cv.medianBlur(image,21)
    
    kernel = np.ones((19,19), np.uint8)
    eroded = cv.erode(image_median_blur, kernel)
    
    #stacked_image = np.hstack((image, image_median_blur, eroded))
    #show_image('preprocessing', stacked_image)
    
    edges =  cv.Canny(eroded, 0, 0)
    #show_image('edges', edges)
    
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    careu = contours[np.argmax([cv.contourArea(x) for x in contours])]
    #cv.drawContours(original, careu, -1, (0, 0, 255), 10)
    #show_image('careu', original)
    top_left = None
    bottom_right = None
    for point in careu.squeeze():
        if top_left is None or point[0] + point[1] < top_left[0] + top_left[1]:
            top_left = point
        if bottom_right is None or point[0] + point[1] > bottom_right[0] + bottom_right[1] :
            bottom_right = point
    
    diff = np.diff(careu.squeeze(), axis = 1)
    top_right = careu.squeeze()[np.argmin(diff)]
    bottom_left = careu.squeeze()[np.argmax(diff)]
    
    #cv.circle(original,tuple(top_left),20,(0,0,255),-1)
    #cv.circle(original,tuple(top_right),20,(0,255,0),-1)
    #cv.circle(original,tuple(bottom_left),20,(255,0,0),-1)
    #cv.circle(original,tuple(bottom_right),20,(0,0,0),-1)
    #show_image("detected corners",original)
    
    square_len = 150 
    size = 150 * 15 + 60

    puzzle = np.array([top_left-30, top_right+[30,-30], bottom_right+30, bottom_left+[-30,30]], dtype="float32")
    destination_of_puzzle  = np.array([[0,0],[size,0], [size, size],[0, size]], dtype="float32")
    M=cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
    result = cv.warpPerspective(original, M, (size, size))
    #show_image('result', result)
    
    return result

In [4]:
def evidentiere_piese(image):
    low_yellow = (0, 110, 0)
    high_yellow = (255, 255, 255)
    img_hsv = cv.cvtColor(image , cv.COLOR_BGR2HSV)
    image = cv.inRange(img_hsv, low_yellow, high_yellow)

    image_median_blur = cv.medianBlur(image,9)
    #show_image('evidentiere_piese', image_median_blur)
    return image_median_blur

In [5]:
horizontal_lines = []
for i in range(30, 30+15*150+1, 150):
    l=[]
    l.append((30, i))
    l.append((30+15*150-1,i))
    horizontal_lines.append(l)

In [6]:
vertical_lines = []
for i in range(30, 30+15*150+1, 150):
    l=[]
    l.append((i, 30))
    l.append((i, 30+15*150-1))
    vertical_lines.append(l)

In [7]:
def afisare_linii(image):
    for line in  vertical_lines : 
        cv.line(image, line[0], line[1], (0, 255, 0), 3)
    for line in  horizontal_lines : 
        cv.line(image, line[0], line[1], (0, 0, 255), 3)
    show_image('img', image)

In [8]:
careuInitial = extrage_careu(cv.imread('imagini_auxiliare/01.jpg'))

In [9]:
def identificare_coordonate(imagineaAnterioara, imagineaCurenta):
    original = imagineaCurenta.copy()
    imagineaAnterioara = evidentiere_piese(imagineaAnterioara)
    imagineaCurenta = evidentiere_piese(imagineaCurenta)
    #show_image('starea anterioara', imagineaAnterioara)
    #show_image('starea curenta', imagineaCurenta)

    diff = cv.absdiff(imagineaAnterioara, imagineaCurenta)
    #show_image('dif', diff)

    max1 = [0, None]    
    max2 = [0, None]

    for i in range(len(horizontal_lines)-1):
        for j in range(len(vertical_lines)-1):
            y_min = horizontal_lines[i][0][1] 
            y_max = horizontal_lines[i + 1][0][1]
            x_min = vertical_lines[j][0][0] 
            x_max = vertical_lines[j + 1][0][0] 
            patch = diff[y_min:y_max, x_min:x_max]
            suma = np.sum(patch)
            #show_image('patch', patch)
            if suma > max1[0]:
                max2 = max1
                max1 = [suma, [x_min, y_min]]
            elif suma > max2[0]:
                max2 = [suma, [x_min, y_min]]

    max1, max2 = max1[1], max2[1]
    #cv.rectangle(original, (max1[0], max1[1]), (max1[0]+150, max1[1]+150), color=(0, 0, 255), thickness=5)
    #cv.rectangle(original, (max2[0], max2[1]), (max2[0]+150, max2[1]+150), color=(0, 255, 0), thickness=5)
    #show_image('piesa noua', original)

    return [max1, max2]

In [10]:
letters = [chr(i) for i in range(ord('A'), ord('O') + 1)]
#print(letters)

In [11]:
def proceseaza_piesa(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(image, 150, 255, cv.THRESH_BINARY)

    image_median_blur = cv.medianBlur(thresh, 3)
 
    kernel = np.ones((3,3), np.uint8)
    dilated = cv.dilate(image_median_blur, kernel)
    #stacked_image = np.hstack((thresh, image_median_blur, dilated))
    #show_image('procesare', stacked_image)
 
    return dilated

In [12]:
def extrage_jumatati(piesa, portiune_linie):
    x_min, x_max, y_min, y_max = portiune_linie
    portiune_linie = piesa[y_min:y_max, x_min:x_max].copy()
    #show_image('portiune_linie', portiune_linie)
    original = portiune_linie.copy()
    
    portiune_linie = cv.cvtColor(portiune_linie, cv.COLOR_BGR2GRAY)
    image_median_blur = cv.medianBlur(portiune_linie,5)
    _, thresh = cv.threshold(image_median_blur, 150, 255, cv.THRESH_BINARY)
    
    #stacked_image = np.hstack((portiune_linie, image_median_blur, thresh))
    #show_image('prelucrare portiune linie', stacked_image)
    
    edges =  cv.Canny(thresh ,200,400)
    #show_image('edges',edges)

    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    linie = contours[np.argmax([cv.contourArea(x) for x in contours])]
    
    top_left = None
    bottom_right = None
    for point in linie.squeeze():
        if top_left is None or point[0] + point[1] < top_left[0] + top_left[1]:
            top_left = point
        if bottom_right is None or point[0] + point[1] > bottom_right[0] + bottom_right[1] :
            bottom_right = point
    
    diff = np.diff(linie.squeeze(), axis = 1)
    top_right = linie.squeeze()[np.argmin(diff)]
    bottom_left = linie.squeeze()[np.argmax(diff)]
    
    #contur linie
    #cv.circle(original,tuple(top_left),20,(0,0,255),-1)
    #cv.circle(original,tuple(top_right),20,(0,255,0),-1)
    #cv.circle(original,tuple(bottom_left),20,(255,0,0),-1)
    #cv.circle(original,tuple(bottom_right),20,(0,0,0),-1)
    #show_image("colturi linie", original)

    top_left += [x_min, y_min]  
    top_right += [x_min, y_min] 
    bottom_left += [x_min, y_min] 
    bottom_right += [x_min, y_min]
 
    #contur linie in imaginea completa 
    #cv.circle(piesa,tuple(top_left),10,(0,0,255),-1)
    #cv.circle(piesa,tuple(top_right),10,(0,255,0),-1)
    #cv.circle(piesa,tuple(bottom_left),10,(255,0,0),-1)
    #cv.circle(piesa,tuple(bottom_right),10,(0,0,0),-1)
    #show_image("colturi linie in piesa", piesa) 

    try:
        if piesa.shape[0] < piesa.shape[1]:
            jumatate1 = piesa[top_left[1]-15:bottom_left[1]+15, top_left[0]-150:top_left[0]-2]
            jumatate2 = piesa[top_left[1]-15:bottom_left[1]+15, top_right[0]+2:top_right[0]+150]
        else:
            jumatate1 = piesa[top_left[1]-150:top_left[1]-2, top_left[0]-15:top_right[0]+15]
            jumatate2 = piesa[bottom_left[1]+2:bottom_left[1]+150, top_left[0]-15:top_right[0]+15]
        
        #show_image("jumatatea 1", jumatate1)
        #show_image("jumatatea 2", jumatate2)
        
        if jumatate1.shape[0] == 0 or jumatate1.shape[1] == 0 or jumatate2.shape[0] == 0 or jumatate2.shape[1] == 0:
            return[[],[]]
    except:
        return [[],[]]
    
    return [jumatate1, jumatate2]

In [13]:
def identificare_numere(img, coord):
    piesa, poriune_linie = extrage_piesa(img, coord)
    jumatate1, jumatate2 = extrage_jumatati(piesa, poriune_linie)
    
    if len(jumatate1) == 0 or len(jumatate2) == 0:
        return [0,0]
    
    jumatate1 = proceseaza_piesa(jumatate1)
    jumatate2 = proceseaza_piesa(jumatate2)
    
    bestMatch1 = None
    bestMatch2 = None
    values = [None, None]
    
    if(coord[0][0] == coord[1][0]):
        nume_folder = 'verticale_taiate'
    else:
        nume_folder = 'orizontale_taiate'
        
    for i in range(0, 7):
        template = cv.imread(f'{nume_folder}/{i}.jpeg')
        template = proceseaza_piesa(template)
        
        kernel = np.ones((2,2), np.uint8)
        template = cv.dilate(template, kernel)
        #show_image('jum1', jumatate1)
        #show_image('template', template)

        height1, width1 = jumatate1.shape
        height2, width2 = jumatate2.shape
        height3, width3 = template.shape
        
        if height1 <= height3 or width1 <= width3 or height2 <= height3 or width2 <= width3:
            return [0,0]
        
        match1 = np.min(cv.matchTemplate(jumatate1, template,  cv.TM_SQDIFF_NORMED))
        match2 = np.min(cv.matchTemplate(jumatate2, template,  cv.TM_SQDIFF_NORMED))
    
        if bestMatch1 == None or bestMatch1 > match1:
            bestMatch1 = match1                        
            values[0] = i
        if bestMatch2 == None or bestMatch2 > match2:
            bestMatch2 = match2                        
            values[1] = i
  
    return values
                

In [14]:
def extrage_piesa(img, coord):
    
    #daca e pe orizontala
    if(coord[0][1] == coord[1][1]):
        x_min = 30 + 100
        x_max = 30 + 150 + 50
        y_min = 0 
        y_max = 150 + 60
        
    #daca e pe verticala
    else: 
        x_min = 0
        x_max = 150 + 60
        y_min = 30 + 100
        y_max = 30 + 150 + 50
        
    piesa = img[coord[0][1]-30:coord[1][1]+150+30, coord[0][0]-30:coord[1][0]+150+30]
    #show_image('piesa', piesa)
    return [piesa, [x_min, x_max, y_min, y_max]]

In [15]:
#matricea punctelor de pe tabla
m = np.zeros((15, 15), dtype=int)

m[0][0] = m[14][14] = m[0][14] = m[14][0] = 5
m[0][3] = m[0][11] = m[1][5] = m[1][9] = m[3][0] = m[11][0] = m[5][1] = m[9][1] = m[14][3] = m[14][11] = m[13][5] = m[13][9] = m[3][14] = m[11][14] = m[5][13] = m[9][13] = 4
m[0][7] = m[1][2] = m[1][12] = m[2][1] = m[2][13] = m[3][3] = m[3][11] = m[7][0] = m[7][14] = m[11][3] = m[11][11] = m[12][1] = m[12][13] = m[13][2] = m[13][12] = m[14][7] = 3
m[2][4] = m[2][10] = m[3][5] = m[3][9] = m[4][2] = m[4][12] = m[5][3] = m[5][11] = m[9][3] = m[9][11] = m[10][2] = m[10][12] = m[11][5] = m[11][9] = m[12][4] = m[12][10] = 2
m[4][4] = m[4][6] = m[4][8] = m[4][10] = m[5][5] = m[5][9] = m[6][4] = m[6][10] = m[8][4] = m[8][10] = m[9][5] = m[9][9] = m[10][4] = m[10][6] = m[10][8] = m[10][10] = 1

In [16]:
#traseul de pe margine
margine = [1, 2, 3, 4, 5, 6, 0, 2, 5, 3, 4, 6, 2, 2, 0, 3, 5, 4, 1, 6, 2, 4, 5, 5, 0, 6, 3, 4, 2, 0, 1, 5, 1, 3, 4, 4, 4, 5, 0, 6, 3, 5, 4, 1, 3, 2, 0, 0, 1, 1, 2, 3, 6, 3, 5, 2, 1, 0, 6, 6, 5, 2, 1, 2, 5, 0, 3, 3, 5, 0, 6, 1, 4, 0, 6, 3, 5, 1, 4, 2, 6, 2, 3, 1, 6, 5, 6, 2, 0, 4, 0, 1, 6, 4, 4, 1, 6, 6, 3, 0]

In [17]:
def calculeaza_scor(scor_jucatori, coord, numere, pozitie_jucatori, jucator_curent):
    scor = [0,0]
    scor[jucator_curent] = m[coord[0][0]-1][coord[0][1]-1] + m[coord[1][0]-1][coord[1][1]-1]
    if(numere[0] == numere[1]):
        scor[jucator_curent] *= 2
        
    if pozitie_jucatori[0] != -1 and (margine[pozitie_jucatori[0]] == numere[0] or margine[pozitie_jucatori[0]] == numere[1]):
        scor[0] += 3
    if pozitie_jucatori[1] != -1 and (margine[pozitie_jucatori[1]] == numere[0] or margine[pozitie_jucatori[1]] == numere[1]):
        scor[1] += 3    
        
    pozitie_jucatori[0] += scor[0]
    pozitie_jucatori[1] += scor[1]
    scor_jucatori[0] += scor[0]
    scor_jucatori[1] += scor[1]
    
    return scor[jucator_curent], scor_jucatori, pozitie_jucatori

In [18]:
def run(input_folder_name):
    for joc in range(1,6):
        imagineaAnterioara = careuInitial.copy()
        scor_jucatori = [0,0]
        pozitie_jucatori = [-1, -1]
        mutari = []
        with open(f'{input_folder_name}/{joc}_mutari.txt', 'r') as file:
            for line in file:
                if not line.isspace():
                    mutari += [int(line[-2])-1]

        for image in range(1,21): 
            img = cv.imread(f'{input_folder_name}/{joc}_{image:02d}.jpg')
            img = extrage_careu(img)
                                
            coord = identificare_coordonate(imagineaAnterioara, img)
            imagineaAnterioara = img
            
            coord = sorted(coord, key=itemgetter(0,1))
            coord_afisare = [[coord[0][0]//150+1, coord[0][1]//150+1], [coord[1][0]//150+1, coord[1][1]//150+1]]
           
            numere = identificare_numere(img, coord)

            jucator_curent = mutari[image-1]
            scor_curent, scor_jucatori, pozitie_jucatori = calculeaza_scor(scor_jucatori, coord_afisare, numere, pozitie_jucatori, jucator_curent)

            coord_afisare[0][0], coord_afisare[1][0] = letters[coord_afisare[0][0]-1], letters[coord_afisare[1][0]-1]
            
            with open(f'352_Cioclov_Maria/{joc}_{image:02d}.txt', 'w') as file:
                file.write(f'{coord_afisare[0][1]}{coord_afisare[0][0]} {numere[0]}\n{coord_afisare[1][1]}{coord_afisare[1][0]} {numere[1]}\n{scor_curent}')
          

In [19]:
run('antrenare')